[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/XPretrain-colab/blob/main/test.ipynb)

In [ ]:
!wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/cut_videos.py
!wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/download_videos.py

!pip install datasets

from datasets import load_dataset
dataset = load_dataset("camenduru/microsoft-XPretrain")

In [ ]:
dataset